In [16]:
import requests
!pip install bs4
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import pandas as pd
import json
import re

In [10]:
url = 'https://litnet.com/ru/top/all'
page = requests.get(url)
page

<Response [200]>

In [11]:
soup = BeautifulSoup(page.text, 'html')
print(soup.prettify())

<!DOCTYPE html>
<html lang="ru-RU">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=970" name="viewport"/>
  <meta content="970" name="MobileOptimized"/>
  <meta content="3abf4baff46cf587" name="yandex-verification"/>
  <link href="/common/img/pwa/ios/256.png" rel="apple-touch-icon"/>
  <link href="/common/img/pwa/ios/72.png" rel="apple-touch-icon" sizes="72x72"/>
  <link href="/common/img/pwa/ios/114.png" rel="apple-touch-icon" sizes="114x114"/>
  <link href="/common/img/pwa/ios/144.png" rel="apple-touch-icon" sizes="144x144"/>
  <link href="/manifest.webmanifest" rel="manifest"/>
  <link href="/manifest.json" rel="manifest"/>
  <meta content="Читать книги онлайн. Подборка популярных произведений: рейтинги, отзывы, бесплатное чтение на сайте Litnet." name="description"/>
  <meta content="_csrf" name="csrf-param"/>
  <meta content="hEJC-nhvtelXyd0bEb3H0blHiwrFdUiF1uuNspBXMALNenSRCQXiuBuaui533oXjwQ7BP5AMJeCYqsjt4TZvYw==" name="csrf-token"/>
  <link href="https://fonts.gstatic.c

In [12]:
url0 = 'https://litnet.com/ru/book/bespridannica-iskra-knyazya-b552634'
page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text, 'html')
print(soup0.prettify())

<!DOCTYPE html>
<html lang="ru-RU">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=970" name="viewport"/>
  <meta content="970" name="MobileOptimized"/>
  <meta content="3abf4baff46cf587" name="yandex-verification"/>
  <link href="/common/img/pwa/ios/256.png" rel="apple-touch-icon"/>
  <link href="/common/img/pwa/ios/72.png" rel="apple-touch-icon" sizes="72x72"/>
  <link href="/common/img/pwa/ios/114.png" rel="apple-touch-icon" sizes="114x114"/>
  <link href="/common/img/pwa/ios/144.png" rel="apple-touch-icon" sizes="144x144"/>
  <link href="/manifest.webmanifest" rel="manifest"/>
  <link href="/manifest.json" rel="manifest"/>
  <meta content="НОВИНКА ПО МИРУ ИСКР!САМОСТОЯТЕЛЬНАЯ ИСТОРИЯ! 

ГРАФИК ПРОД в ноябре: через день (до 5.11 по нечетным, с 6.11 по четным)." name="description"/>
  <meta content="_csrf" name="csrf-param"/>
  <meta content="auCU0EuFgf2Wc9Ry7gTuI7GofeQ8Y9dOJ5CllwGf-6UvrsWVCODGreUagj3eT4Jl9-Mnj3M1uQxJ_PLvVPaX_A==" name="csrf-token"/>
  <link href="https://f

In [14]:
#<a href="/ru/book/60-letnyaya-istinnaya-dlya-400-letnego-drakona-b556938">
unique_urls = set()
for i in tqdm(range(5)): #!!!потом вместо 5 нужно ввести колличество страниц которое будем парсить!!!
    url = 'https://litnet.com/ru/top/all?alias=all&page=' + str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')
    for link in soup.find_all('a'):
        a = link.get('href')
        if a is not None and '/ru/book' in a:
            unique_urls.add('https://litnet.com' + a)
    sleep(7)
# for url in unique_urls:
#     print(url)

100%|██████████| 5/5 [00:41<00:00,  8.33s/it]


In [18]:
book_data = []
for url in tqdm(unique_urls, desc="Парсинг книг"):
    try:
        page = requests.get(url, timeout=10)
        if page.status_code != 200:
            continue
        soup = BeautifulSoup(page.text, 'html.parser')
        #Основные данные
        title_tag = soup.find('h1', class_='roboto')
        title = title_tag.get_text(strip=True) if title_tag else None
        authors = []
        author_tags = soup.select('h2.p a.author')
        for a in author_tags:
            authors.append(a.get_text(strip=True))
        authors_str = ', '.join(authors) if authors else None
        #Жанры
        genre_tags = soup.select('.breadcrumb li a')
        genres = [g.get_text(strip=True) for g in genre_tags][1:]  # пропускаем "Литнет"
        genres_str = ', '.join(genres) if genres else None
        #Аннотация
        annotation_div = soup.find('div', id='annotation')
        annotation = annotation_div.get_text(strip=True) if annotation_div else None
        #Цикл
        cycle_tag = soup.find('span', string=re.compile(r'Цикл:'))
        cycle = cycle_tag.find_next('a').get_text(strip=True) if cycle_tag else None
        #Возрастное ограничение
        age_span = soup.find('span', class_='age-restriction-label')
        age = age_span.get_text(strip=True).replace('Ограничение:', '').strip() if age_span else None
        #Рейтинг и статистика
        rating_value = soup.find('span', class_='book-rating-info-value')
        rating = rating_value.get_text(strip=True).split()[0] if rating_value else None
        likes_span = soup.find('a', class_='rate-btn-like')
        likes = likes_span.find('span').get_text(strip=True) if likes_span and likes_span.find('span') else None
        favourites = soup.find('span', class_='count-favourites')
        favourites_count = favourites.get_text(strip=True) if favourites else None
        views = soup.find('span', class_='count-views')
        views_count = views.get_text(strip=True) if views else None
        #Дата публикации (из meta или видимого блока)
        pub_meta = soup.find('span', itemprop='datePublished')
        publication_date = pub_meta.get('content', '').strip() if pub_meta else None
        #Цена или тип доступа
        price_span = soup.find('span', class_='ln_btn-get-text')
        price_or_access = price_span.get_text(strip=True) if price_span else None
        #Статус книги
        status_span = soup.find('span', class_='book-status-process')
        status = status_span.get_text(strip=True) if status_span else None
        # Сохранение
        book_data.append({
            'url': url,
            'title': title,
            'authors': authors_str,
            'genres': genres_str,
            'cycle': cycle,
            'age_restriction': age,
            'annotation': annotation,
            'rating': rating,
            'likes': likes,
            'favourites_count': favourites_count,
            'views_count': views_count,
            'publication_date': publication_date,
            'price_or_access': price_or_access,
            'status': status
        })

        sleep(5)
    except Exception as e:
        print(f"Ошибка при парсинге {url}: {e}")

Парсинг книг: 100%|██████████| 80/80 [08:24<00:00,  6.31s/it]


In [19]:
df = pd.DataFrame(book_data)
display(df.head())
display(df.tail())
df.to_csv('litnet_books_dataset.csv', index=False, encoding='utf-8-sig')

,url,title,authors,genres,cycle,age_restriction,annotation,rating,likes,favourites_count,views_count,publication_date,price_or_access,status
0,https://litnet.com/ru/book/iskushaya-sudbu-ya-...,Искушая судьбу. Я тебя не жду,Сара Адам,"Любовные романы, Остросюжетный любовный роман",Джон и Адалин,None,"Аннотация к книге ""Искушая судьбу. Я тебя не ж...",1099,985,4389,400817,2025-11-08,Подписка 139 RUB,В процессе: 08 Ноя170 стр.
1,https://litnet.com/ru/book/ya-vybirayu-tebya-b...,Я выбираю тебя,Марина Кистяева,"Эротика, Романтическая эротика",Мы выбираем...,None,"Аннотация к книге ""Я выбираю тебя""Иногда прошл...",2313,2107,10947,399080,2025-11-07,Подписка 139 RUB,В процессе: 07 Ноя120 стр.
2,https://litnet.com/ru/book/tvoi-pervyi-budet-b...,"Твой первый. Будет больно, но тебе понравится",Иман Кальби,"Любовные романы, Остросюжетный любовный роман",None,None,"Аннотация к книге ""Твой первый. Будет больно, ...",987,895,6876,134662,2025-11-07,None,В процессе: 07 Ноя47 стр.
3,https://litnet.com/ru/book/zhena-po-obmanu-b52...,Жена по обману,Екатерина Флат,"Фэнтези, Юмористическое фэнтези",None,None,"Аннотация к книге ""Жена по обману""Попасть в др...",1539,1468,6651,1078072,2025-11-05,Купить 199 RUB,None
4,https://litnet.com/ru/book/posle-razvoda-desya...,После развода. Десять лет спустя,Анна Томченко,"Любовные романы, Романы о неверности",None,None,"Аннотация к книге ""После развода. Десять лет с...",2396,2324,12094,1412829,2025-11-09,Подписка 139 RUB,В процессе: 09 Ноя241 стр.


,url,title,authors,genres,cycle,age_restriction,annotation,rating,likes,favourites_count,views_count,publication_date,price_or_access,status
75,https://litnet.com/ru/book/nenavizhu-lyublyu-t...,Ненавижу... люблю тебя,Валерия Ангелос,"Любовные романы, Криминальный любовный роман",Горячий опасный герой,None,"Аннотация к книге ""Ненавижу... люблю тебя""— Не...",5322,5020,28922,1441272,2025-11-06,Подписка 129 RUB,В процессе: 06 Ноя140 стр.
76,https://litnet.com/ru/book/sluzhanka-rektora-a...,Служанка ректора Академии военных драконов,Матильда Аваланж,"Попаданцы, Попаданцы в другие миры",None,None,"Аннотация к книге ""Служанка ректора Академии в...",2529,2354,17837,1343347,2025-11-09,None,В процессе: 09 Ноя161 стр.
77,https://litnet.com/ru/book/otvergnutaya-nevest...,Отвергнутая невеста. Целительница на краю импе...,Виктория Богачева,"Фэнтези, Боевое фэнтези",None,None,"Аннотация к книге ""Отвергнутая невеста. Целите...",1534,1403,10340,538697,2025-11-09,Подписка 139 RUB,В процессе: 09 Ноя190 стр.
78,https://litnet.com/ru/book/ne-rodnaya-mama-ona...,( Не) родная мама. Она - мой смысл жизни,Виктория Вишневская,"Любовные романы, Служебный роман",None,None,"Аннотация к книге ""( Не) родная мама. Она - мо...",1252,1199,7430,179197,2025-11-08,None,В процессе: 08 Ноя45 стр.
79,https://litnet.com/ru/book/razvod-vernu-tebya-...,Развод. Верну тебя любой ценой,"Оксана Барских, Оксана Литвинова","Любовные романы, Романы о неверности",None,None,"Аннотация к книге ""Развод. Верну тебя любой це...",1501,1497,11841,332176,2025-11-07,None,В процессе: 07 Ноя50 стр.
